# Clean FreeSurfer -3T Output

We need to remove all the subjects from the folder that failed, because otherwise mindcontrol will not be happy.

In [1]:
## Setup
from pathlib import Path
import pandas as pd
import numpy as np
import subprocess
from datetime import date
import re
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

In [4]:
nndsp_jobfails = !jobhist 6902333 | grep "FAILED"
nki_jobfails = !jobhist 6456604 | grep "FAILED"

In [20]:
nndsp_fs_output = Path('/data/NNDSP/derivatives/fs_nndsp_3T')
nki_fs_output = Path('/data/NNDSP/derivatives/fs_nki_3T')

nndsp_swarm = Path('/data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_09')
nki_swarm = Path('/data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_02')

In [16]:
nndsp_numfails = []
for l in nndsp_jobfails:
    nndsp_numfails.append(re.match(r'6902333_(\d+)\s', l).group(0).split(' ')[0])

In [17]:
nki_numfails = []
for l in nki_jobfails:
    nki_numfails.append(re.match(r'6456604_(\d+)\s', l).group(0).split(' ')[0])

## NNDSP 

### Get all the Subjects that Failes

In [28]:
import re
import sys

failed_subjects = []
for failed in nndsp_numfails:
    file_path = nndsp_swarm.joinpath('swarm_'+str(failed)+'.o')
    print(file_path.is_file(), file_path)
    
    with open(file_path, 'r') as swarm_file:
        print("Reading File")
        data=swarm_file.read().replace('\n', '')
        matches = [m.group(0).split(' ')[1] for m in re.finditer('participant_label\s(\d|(a-z))+\s', data)]
        
        for sub in matches:
            failed_subjects.append(sub)
            
        print(matches)

True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_09/swarm_6902333_6.o
Reading File
['0150']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_09/swarm_6902333_19.o
Reading File
['0208']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_09/swarm_6902333_25.o
Reading File
['0233']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_09/swarm_6902333_34.o
Reading File
['0284']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_09/swarm_6902333_49.o
Reading File
['0425']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_09/swarm_6902333_92.o
Reading File
['0682']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_09/swarm_6902333_103.o
Reading File
['0775']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_09/swarm_6902333_151.o
Reading File
['1043']
True /data/NNDS

In [29]:
len(failed_subjects)

25

### Get all Failed Directories Removed

In [30]:
df_subs = pd.DataFrame([p.as_posix() for p in nndsp_fs_output.glob('sub-*')], columns=['subj_path'])
df_subs.head()

,subj_path
0,/data/NNDSP/derivatives/fs_nndsp_3T/sub-1889
1,/data/NNDSP/derivatives/fs_nndsp_3T/sub-2011
2,/data/NNDSP/derivatives/fs_nndsp_3T/sub-1948
3,/data/NNDSP/derivatives/fs_nndsp_3T/sub-1198
4,/data/NNDSP/derivatives/fs_nndsp_3T/sub-1371


In [32]:
df_subs = df_subs.assign(subject = [Path(p).name[4:] for p in df_subs.subj_path])
df_subs = df_subs.sort_values(by='subject')
df_subs.head()

,subj_path,subject
53,/data/NNDSP/derivatives/fs_nndsp_3T/sub-0102,0102
117,/data/NNDSP/derivatives/fs_nndsp_3T/sub-0114,0114
333,/data/NNDSP/derivatives/fs_nndsp_3T/sub-0126,0126
295,/data/NNDSP/derivatives/fs_nndsp_3T/sub-0127,0127
140,/data/NNDSP/derivatives/fs_nndsp_3T/sub-0133,0133


In [33]:
df_failed = pd.DataFrame(failed_subjects, columns = ['subject'])
df_failed.head()

,subject
0,0150
1,0208
2,0233
3,0284
4,0425


In [34]:
df_failed = pd.merge(df_failed, df_subs, on = 'subject')

In [35]:
print(df_failed.shape)
df_failed.head()

(25, 2)


,subject,subj_path
0,0150,/data/NNDSP/derivatives/fs_nndsp_3T/sub-0150
1,0208,/data/NNDSP/derivatives/fs_nndsp_3T/sub-0208
2,0233,/data/NNDSP/derivatives/fs_nndsp_3T/sub-0233
3,0284,/data/NNDSP/derivatives/fs_nndsp_3T/sub-0284
4,0425,/data/NNDSP/derivatives/fs_nndsp_3T/sub-0425


#### Remove all Failed Directories

In [36]:
import shutil
import os

for path in df_failed.subj_path:
        shutil.rmtree(path)

## NKI

In [44]:
import re
import sys

failed_subjects = []
for failed in nki_numfails:
    file_path = nki_swarm.joinpath('swarm_'+str(failed)+'.o')
    print(file_path.is_file(), file_path)
    
    with open(file_path, 'r') as swarm_file:
        print("Reading File")
        data=swarm_file.read().replace('\n', '')
        matches = [m.group(0).split(' ')[1] for m in re.finditer('participant_label\s(\d|(A))+\s', data)]
        
        for sub in matches:
            failed_subjects.append(sub)
            
        print(matches)

True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_02/swarm_6456604_913.o
Reading File
['A00074034']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_02/swarm_6456604_0.o
Reading File
['A00008326']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_02/swarm_6456604_48.o
Reading File
['A00028784']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_02/swarm_6456604_67.o
Reading File
['A00029231']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_02/swarm_6456604_74.o
Reading File
['A00030980']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_02/swarm_6456604_157.o
Reading File
['A00035440']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_02/swarm_6456604_161.o
Reading File
['A00035561']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_02/swarm_6456604_226.o


True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_02/swarm_6456604_891.o
Reading File
['A00066698']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_02/swarm_6456604_894.o
Reading File
['A00066784']
True /data/NNDSP/anal/freesurfer_files/other_files/swarm_output/swarm_/2018_08_02/swarm_6456604_901.o
Reading File
['A00066860']


In [45]:
len(failed_subjects)

70

In [48]:
### Get all Failed Directories Removed
df_subs = pd.DataFrame([p.as_posix() for p in nki_fs_output.glob('sub-*')], columns=['subj_path'])
df_subs = df_subs.assign(subject = [Path(p).name[4:13] for p in df_subs.subj_path])
df_subs = df_subs.sort_values(by='subject')
df_subs.head()

,subj_path,subject
1458,/data/NNDSP/derivatives/fs_nki_3T/sub-A0000832...,A00008326
1548,/data/NNDSP/derivatives/fs_nki_3T/sub-A0000839...,A00008399
2847,/data/NNDSP/derivatives/fs_nki_3T/sub-A0000839...,A00008399
664,/data/NNDSP/derivatives/fs_nki_3T/sub-A00008399,A00008399
2253,/data/NNDSP/derivatives/fs_nki_3T/sub-A0001089...,A00010893


In [49]:
df_failed = pd.DataFrame(failed_subjects, columns = ['subject'])
df_failed.head()

,subject
0,A00074034
1,A00008326
2,A00028784
3,A00029231
4,A00030980


In [50]:
df_failed = pd.merge(df_failed, df_subs, on = 'subject')
print(df_failed.shape)
df_failed.head()

(95, 2)


,subject,subj_path
0,A00074034,/data/NNDSP/derivatives/fs_nki_3T/sub-A0007403...
1,A00008326,/data/NNDSP/derivatives/fs_nki_3T/sub-A0000832...
2,A00028784,/data/NNDSP/derivatives/fs_nki_3T/sub-A00028784
3,A00028784,/data/NNDSP/derivatives/fs_nki_3T/sub-A0002878...
4,A00029231,/data/NNDSP/derivatives/fs_nki_3T/sub-A0002923...


In [51]:
#### Remove all Failed Directories

import shutil
import os

for path in df_failed.subj_path:
    print(path)    
    shutil.rmtree(path)

/data/NNDSP/derivatives/fs_nki_3T/sub-A00074034_ses-CLGA
/data/NNDSP/derivatives/fs_nki_3T/sub-A00008326_ses-DS2
/data/NNDSP/derivatives/fs_nki_3T/sub-A00028784
/data/NNDSP/derivatives/fs_nki_3T/sub-A00028784_ses-DS2
/data/NNDSP/derivatives/fs_nki_3T/sub-A00029231_ses-DS2
/data/NNDSP/derivatives/fs_nki_3T/sub-A00030980_ses-DS2
/data/NNDSP/derivatives/fs_nki_3T/sub-A00035440_ses-DS2
/data/NNDSP/derivatives/fs_nki_3T/sub-A00035561_ses-DS2
/data/NNDSP/derivatives/fs_nki_3T/sub-A00037877_ses-DS2
/data/NNDSP/derivatives/fs_nki_3T/sub-A00038605_ses-DS2
/data/NNDSP/derivatives/fs_nki_3T/sub-A00039657_ses-DS2
/data/NNDSP/derivatives/fs_nki_3T/sub-A00040623
/data/NNDSP/derivatives/fs_nki_3T/sub-A00040623_ses-DS2
/data/NNDSP/derivatives/fs_nki_3T/sub-A00040800_ses-CLG2
/data/NNDSP/derivatives/fs_nki_3T/sub-A00040800_ses-CLG4
/data/NNDSP/derivatives/fs_nki_3T/sub-A00040800_ses-CLG2R
/data/NNDSP/derivatives/fs_nki_3T/sub-A00040815_ses-CLG2
/data/NNDSP/derivatives/fs_nki_3T/sub-A00040815_ses-CLG4
/